In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col, when, lit, upper, trim
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, sum as sf_sum, count


session = get_active_session()


In [ ]:
session.use_database("IPL")
session.use_schema("IPL_SCHEMA")
session.use_warehouse("BIGDATA_IPL")

In [ ]:
df_main = session.table("BIGDATA_IPL")
df_main.show(5)


row_count = df_main.count()
print(f"Total rows: {row_count}\n")

In [ ]:
print("Schema of df_main:\n")
print(df_main.schema)


In [ ]:
print("Columns in df_main:")
print(df_main.columns)


In [ ]:
print("STEP: Top Bowlers by Economy Rate (df_top_bowlers)")

df_top_bowlers = (
    df_main.group_by("bowler")
           .agg(
               sf_sum(col("total_runs")).alias("RUNS_CONCEDED"),
               count("*").alias("BALLS")
           )
           .filter(col("BALLS") > 300)
           .with_column("ECONOMY", col("RUNS_CONCEDED") * 6 / col("BALLS"))
           .sort(col("ECONOMY"))
           .limit(10)
)

df_top_bowlers.show()

df_top_bowlers


In [ ]:
print(" STEP: Top 10 Batsmen by Runs (df_top_batsmen) ")

df_top_batsmen = (
    df_main.group_by("batsman")
           .agg({"batsman_runs": "sum"})
           .sort(col("SUM(BATSMAN_RUNS)").desc())
           .limit(10)
)

df_top_batsmen.show()

df_top_batsmen


In [ ]:
print(" STEP: Team-wise Scoring Summary (df_team_stats) ")

df_team_stats = (
    df_main.group_by("batting_team")
           .agg({"total_runs": "sum", "match_id": "count"})
           .with_column_renamed("SUM(TOTAL_RUNS)", "TOTAL_RUNS")
           .with_column_renamed("COUNT(MATCH_ID)", "BALLS")
           .sort(col("TOTAL_RUNS").desc())
)

df_team_stats.show()

df_team_stats
